In [11]:
%matplotlib qt
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from matplotlib.patches import Patch
from cmap import Colormap
from matplotlib.colors import LinearSegmentedColormap
import nibabel as nib
from nilearn.plotting import (plot_stat_map,
                                plot_surf_stat_map,
                                plot_roi,
                                show
                                )
import nibabel.freesurfer.io as fsio
from scipy.stats import ttest_ind
import pingouin as pg
from scipy.spatial import cKDTree
from scipy.spatial.distance import pdist, squareform
from scipy.stats import wasserstein_distance
from nilearn.datasets import fetch_surf_fsaverage
from mne import read_labels_from_annot
from mne.viz import Brain
from matplotlib.colors import ListedColormap
from neuromaps.datasets import fetch_fslr
from surfplot import Plot
from brainspace.datasets import load_parcellation
from sklearn.manifold import MDS
from brainspace.gradient import GradientMaps
from brainspace.utils.parcellation import map_to_labels

ssa_dir = Path("./material/with_qc/SSA")

Voxel Based Morphometry

In [ ]:
img_dir = Path.cwd() / "material" / "with_qc" / "VBM" / "stats"
bg_image = "/Users/payamsadeghishabestari/fsl/data/standard/MNI152_T1_2mm.nii.gz"
fname_sub_mask = img_dir / "subcortical_mask_thr80.nii.gz"
fname_1 =  img_dir / "fslvbm_s3_80_tfce_corrp_tstat1.nii.gz"
fname_2 = img_dir / "fslvbm_s3_80_tfce_corrp_tstat2.nii.gz"
bg_image_1 = "/Users/payamsadeghishabestari/fsl/data/standard/MNI152_T1_1mm_brain.nii.gz"
bg_image_2 = "/Users/payamsadeghishabestari/fsl/data/standard/MNI152_T1_0.5mm.nii.gz"
threshold = 0.95
kwargs = {
            "colorbar": True,
            "cbar_tick_format": "%.2g",
            "annotate": False,
            "draw_cross": False,
            "threshold": threshold,
            "radiological": False,
            "cmap": 'autumn',
            "symmetric_cbar": False,
            "vmin": threshold,
            "vmax": 1,
            "dim": -0.3,
            "black_bg": False
        }

fig1 = plot_stat_map(stat_map_img=fname_1, bg_img=bg_image_1, display_mode="xz", cut_coords=None, **kwargs) # CO > TI
fig2 = plot_stat_map(stat_map_img=fname_2, bg_img=bg_image_1, display_mode="x", cut_coords=(-18, -21, -24, -27, -30), **kwargs) # TI > CO
fig3 = plot_stat_map(stat_map_img=fname_2, bg_img=bg_image_1, display_mode="z", cut_coords=(-8, -4, 0, 4, 8), **kwargs) # TI > CO

for idx, fig in enumerate([fig1, fig2, fig3], start=1):
    fig.savefig(ssa_dir.parent / "paper_figures" / "vbm" / f"Figure_{idx}.pdf",
                    dpi=600, bbox_inches='tight')

Surface Based Morphometry (p-maps)

In [ ]:
fsaverage_dir = Path("/Applications/freesurfer/dev/subjects/fsaverage")
root_dir = Path("./material/with_qc/SBM")

labels, ctab, names = fsio.read_annot(fsaverage_dir / "label" / f"rh.aparc.annot")

column_names = [
    'ClusterNo', 'Max', 'VtxMax', 'Size_mm2', 
    'MNIX', 'MNIY', 'MNIZ', 
    'CWP', 'CWPLow', 'CWPHi', 
    'NVtxs', 'WghtVtx', 'Annot'
]
modality = "area"

for hemi, he in zip(["left", "right"], ["lh", "rh"]):
    sulc = fsaverage_dir / "surf" / f"{he}.sulc"
    inflated = fsaverage_dir / "surf" / f"{he}.inflated"
    
    for mod in ["pos", "neg"]:
        stat_map = root_dir / modality / he / "CO-TI-QC" / f"perm.th23.{mod}.sig.cluster.mgz"
        summery_fname = root_dir / modality / he / "CO-TI-QC" / f"perm.th23.{mod}.sig.cluster.summary"

        if mod == "pos":
            input_fname = stat_map

        if mod == "neg": 
            img = nib.load(stat_map)
            data = img.get_fdata()
            data_neg = data * -1
            new_img = nib.Nifti1Image(data_neg, img.affine, img.header)
            input_fname = stat_map.parent / f"perm.th23.{mod}.sig.cluster_inverted.mgz"
            nib.save(new_img, input_fname)
            
        for view in ["medial", "lateral"]:
            figure = plot_surf_stat_map(
                                        stat_map=input_fname,
                                        surf_mesh=inflated,
                                        hemi=hemi,
                                        view=view,
                                        threshold=1.3,
                                        bg_map=sulc,
                                        darkness=0.7,
                                        bg_on_data=False,
                                        vmin=1.3,
                                        vmax=3.89,
                                        colorbar=False,
                                        cmap="autumn"
                                    )
            
            ## if you want to add atlas -> uncomment
            '''
            df = pd.read_csv(summery_fname, skiprows=40, delim_whitespace=True,
                            comment='#', names=column_names)
            labels = df["Annot"].values.tolist()
            levels = [names.index(f"{item}".encode()) for item in labels]

            plot_surf_contours(
                                surf_mesh=fsaverage_dir / "surf" / f"{he}.inflated",
                                roi_map=fsaverage_dir / "label" / f"{he}.aparc.annot",
                                hemi=hemi,
                                labels=labels,
                                levels=levels,
                                legend=False,
                                figure=figure,
                                colors=["cadetblue"] * len(labels),
                            )
            '''

            # figure.figure.set_facecolor('black')
            # figure.axes[0].set_facecolor('black')
            figure.frameon = False
            figure.axes[0].axis('off')
            figure.figure.savefig(Path.cwd() / "material" / "with_qc" / "paper_figures" / "sbm" / modality / f"sbm_sig_{mod}_{hemi}_{view}_nolines_whitebg.pdf",
                                facecolor=figure.figure.get_facecolor(),
                                bbox_inches='tight')
            show()

Surface based morphometry (Find Glasser atlas regions)

In [ ]:
mni_coord = np.array([19.2,  -67.8,   -7.2])
hemi = "rh"
hemi_full = 'right'

fsaverage_dir = Path("/Applications/freesurfer/dev/subjects/fsaverage")
fsaverage = fetch_surf_fsaverage('fsaverage')
surf_path = fsaverage['pial_' + hemi_full]
coords = nib.load(surf_path).darrays[0].data  


labels, ctab, names = fsio.read_annot(fsaverage_dir / "label" / f"{hemi}.HCPMMP1.annot")
kdtree = cKDTree(coords)
dist, vertex_idx = kdtree.query(mni_coord)
label_index = labels[vertex_idx]
label_id = ctab[label_index, -1]

idx = np.where(ctab[:, -1] == label_id)[0]
if len(idx) > 0:
    region_name = names[idx[0]].decode('utf-8')
else:
    region_name = 'Unknown'

print(f"Vertex {vertex_idx} maps to region: {region_name}")
labels = read_labels_from_annot(subject="fsaverage", parc="HCPMMP1", verbose=False)

brain = Brain(
            "fsaverage",
            hemi,
            subjects_dir=None,
            size=(800, 600),
            background="white",
            surf="pial_semi_inflated",
            cortex=["#b8b4ac", "#b8b4ac"]
            )
# brain.add_annotation("HCPMMP1", color="grey")
sel_label = [label for label in labels if label.name == f"{region_name}-{hemi}"][0]
brain.add_label(sel_label, borders=True, color="blue")

ROI analysis

In [2]:
## some functions to read FS files
def load_freesurfer_lut_to_df(lut_path):
    columns = ['Index', 'name', 'R', 'G', 'B', 'A']
    data = []
    with open(lut_path, 'r') as f:
        for line in f:
            if line.startswith('#') or line.strip() == '':
                continue
            parts = line.split()
            if len(parts) >= 6:
                index = int(parts[0])
                label = parts[1]
                rgb = list(map(int, parts[2:5]))
                alpha = int(parts[5])
                data.append([index, label, *rgb, alpha])
    
    return pd.DataFrame(data, columns=columns)

def rgba_to_hex(r, g, b):
    return '#{0:02x}{1:02x}{2:02x}'.format(r, g, b)

In [ ]:
# Load sample subject image 
mri_dir = Path('/Users/payamsadeghishabestari/antinomics_clean_codes/dvob_processed/sMRI/0539/mri')
t1_path = mri_dir / "nu.mgz"
hippo_path = mri_dir / "rh.hippoAmygLabels-T1.v22.mgz"
thalamus_path = mri_dir / "ThalamicNuclei.v13.T1.mgz"
aan_path = mri_dir / "arousalNetworkLabels.v10.mgz"
lut_path = "/Applications/freesurfer/dev/FreeSurferColorLUT.txt"

## make the data label data for nilearn
seg_img = nib.load(aan_path)
seg_data = seg_img.get_fdata().astype(int)
labels = np.unique(seg_data)
label_map = {label: i for i, label in enumerate(labels)}
seg_data_mapped = np.vectorize(label_map.get)(seg_data)
seg_data_4d = seg_data_mapped[..., np.newaxis]
seg_img_4d = nib.Nifti1Image(seg_data_4d, affine=seg_img.affine, header=seg_img.header)
unique_labels = np.unique(seg_data).tolist()

## create lut file for nilearn
lut_df = load_freesurfer_lut_to_df(lut_path)
lut_df['color'] = lut_df.apply(lambda row: rgba_to_hex(row['R'], row['G'], row['B']), axis=1)
lut_df = lut_df.query('Index == @unique_labels')
lut_df.rename(columns={"Index": "index"}, inplace=True)
lut_df = lut_df[["index", "name", "color"]]
lut_df.loc[0, 'name'] = 'Background'
lut_df['index'] = range(len(lut_df))
lut_df.reset_index(drop=True, inplace=True)

figure, ax = plt.subplots(1, 1, figsize=(9, 9), layout="tight")
kwargs = {
        "bg_img": t1_path,                
        "cmap": lut_df,
        "colorbar": False,
        "annotate": False,
        "draw_cross": False,
        "black_bg": True,
        "alpha": 0.7,
        "dim" : -1,
        "figure": figure
        }
plot_roi(
        seg_img_4d,            
        display_mode="y",
        cut_coords=(7,),
        axes=ax,
        **kwargs
        )

legend_handles = [
                Patch(color=row['color'], label=row['name'], alpha=0.7)
                for _, row in lut_df.iterrows()
                ][1:]

fig, ax = plt.subplots()
# fig.patch.set_facecolor('black')
# ax.set_facecolor('black')

legend = ax.legend(handles=legend_handles, title="Regions", facecolor='white', edgecolor='k', frameon=False)

# Set text colors to white
plt.setp(legend.get_texts(), color='k')
plt.setp(legend.get_title(), color='k')

ax.axis('off')
plt.show()

In [26]:
## boxplots
def plot_roi_stats(roi, roi_sel):
    
    ## select what to read
    roi_dir = Path.cwd() / "data" / "roi_stats"
    fname = roi_dir / f"{roi}.csv"
    df = pd.read_csv(fname, index_col=0)

    ## select subjects
    fname_cov = Path.cwd() / "material" / "with_qc" / "behavioural" / "tinception_fsgd_qc.txt"
    df_cov = pd.read_csv(fname_cov, index_col=0, delimiter="\t")
    df_cov.dropna(inplace=True)
    subjects = list(df_cov["Unnamed: 1"])
    df = df.query('subjects == @subjects')
    df["group"] = df_cov["Unnamed: 2"].values

    ## some renaming and fixes
    if "MV(Re)" in df.columns.to_list():
        df = df.rename(columns={"MV(Re)": "MV"})
    df.columns = df.columns.str.replace('-', '_')

    palette_color = ['#1f77b4', '#d62728']

    fig, ax = plt.subplots(1, 1, figsize=(4, 4))
    #sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
    sns.boxplot(data=df, x="group", y=roi_sel,
                palette=palette_color, fill=False, width=0.6,
                gap=0, linewidth=2, ax=ax)
    sns.stripplot(data=df, x="group", y=roi_sel,
                palette=palette_color, linewidth=0, size=3.6,
                edgecolor="k", ax=ax)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_title(roi_sel)
    #ax.set_ylim([150, 440])
    tval, pval = ttest_ind(
                                df.query('group == "CO"')[roi_sel],
                                df.query('group == "TI"')[roi_sel]
                        )
    d = pg.compute_effsize(
                            df.query('group == "CO"')[roi_sel],
                            df.query('group == "TI"')[roi_sel],
                            eftype='cohen'
                            )
    print(f"t(264) = {round(tval, 2)}, p = {round(pval, 3)}, d = {round(d, 2)}")

    return fig

roi = "hipposubfields_lh"
roi_sel = "presubiculum_body"
fig = plot_roi_stats(roi, roi_sel)

t(264) = 1.39, p = 0.167, d = 0.17


/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1741/250748519.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["group"] = df_cov["Unnamed: 2"].values
/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1741/250748519.py:26: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=df, x="group", y=roi_sel,
/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1741/250748519.py:29: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.stripplo

Gradients

In [ ]:
## Plot state space gradients
df_avg_connectome = pd.read_csv("./material/with_qc/SSA/avg_gradients.csv")
cl1 = sns.cubehelix_palette(rot=-.2).as_hex()[2]
cl2 = sns.cubehelix_palette(rot=.2).as_hex()[2]
pal1 = [cl1, cl2]
palette = pal1 + 5 * ["gainsboro"]

y = "Gradient 2"
df_sub = df_avg_connectome.query('alignment == "proc" & method == "dm"')
df_sub['size'] = np.where(df_sub['network'].isin(['Somatomotor', 'Default']), 5, 5)
hue_order = ['Default', 'Somatomotor', 'Dorsal Attention', 'Ventral Attention', 'Limbic', 'Frontoparietal', 'Visual']
# hue_order = ['Somatomotor', 'Limbic', 'Default', 'Dorsal Attention', 'Ventral Attention', 'Frontoparietal', 'Visual']
g = sns.FacetGrid(
                data=df_sub,
                col="group",
                sharey=False,
                sharex="col",
                height=3.9,
                aspect=1.4,
                legend_out=False
                )
g.axes[0][1].spines['left'].set_visible(False)
g.axes[0][1].tick_params(axis='y', left=False, labelleft=False)
g.map_dataframe(
                sns.scatterplot,
                x="Gradient 1",
                y=y,
                hue="network",
                hue_order=hue_order,
                palette=palette,
                #s=15,
                edgecolor="k",
                alpha=0.99,
                )
g.fig.subplots_adjust(wspace=0.2)
g.add_legend(frameon=False, label_order=hue_order[:2])
if y == "Gradient 2":
    g.set(ylim=(-13, 12.5))
if y == "Gradient 3":
    g.set(ylim=(-10, 12))

In [ ]:
## plot histograms
df_hist = df_avg_connectome.query('alignment == "proc" & method == "dm"')
df_hist = pd.melt(
    df_hist,
    id_vars=['group', 'alignment', 'method', 'network'],
    value_vars=['Gradient 1', 'Gradient 2', 'Gradient 3'],
    var_name='Gradient_Label',
    value_name='Gradients'
)
y = "Gradient 1"
networks = ['Default', 'Somatomotor']
df_hist_sub = df_hist.query(f'Gradient_Label == "{y}"') # & network == @networks
palette_color = ['#1f77b4', '#d62728']
g = sns.FacetGrid(data=df_hist_sub, sharex=False, row="Gradient_Label", sharey=False, height=3.9, aspect=1.4)
g.map_dataframe(
                sns.histplot,
                x="Gradients",
                stat="count",
                element="bars",
                multiple="layer",
                bins=300,
                fill=True,
                kde=True,
                line_kws={"lw": 3.5},
                hue="group",
                hue_order=["co", "ti"],
                palette=palette_color,
                discrete=True,
                alpha=0.1,
                linewidth=0.1
)
if y == "Gradient 2":
    g.set(ylim=(-13, 12.5), ylabel="", title="")
if y == "Gradient 3":
    g.set(ylim=(-10, 12), ylabel="", title="")
# g.savefig(ssa_dir.parent / "paper_figures" / "gradients" / f"g1_hist_v2.pdf",
#                 dpi=600, bbox_inches='tight')

In [ ]:
co_hist_vals = df_hist_sub.query("group == 'co'")["Gradients"].to_numpy()
ti_hist_vals = df_hist_sub.query("group == 'ti'")["Gradients"].to_numpy()

emd = wasserstein_distance(co_hist_vals, ti_hist_vals)
print(f"Wasserstein distance: {emd}")

shift_direction = np.mean(ti_hist_vals) - np.mean(co_hist_vals)
print(f"Shift direction (mean_b - mean_a): {shift_direction}")

In [ ]:
## compute brains
datas = []
for group in ["co", "ti"]:
    datas.append(np.load(ssa_dir / f"avg_connectome_{group}.npy", allow_pickle=True))

gm = GradientMaps(
                    n_components=3,
                    approach="dm",
                    kernel=None,
                    alignment="procrustes",
                    random_state=0
                    )
gm.fit(datas)

In [ ]:
labeling = load_parcellation('schaefer', scale=1000, join=True)
mask = labeling != 0
half_sch_idx = 32492
surfaces = fetch_fslr()
lh, rh = surfaces['inflated']
lh_parc, rh_parc = load_parcellation('schaefer', scale=1000)
color = 'dimgrey'
cmap = ListedColormap(color, 'regions', N=1)
color_range_dict1 = {0: (-25, 25),
                    1: (-10, 10),
                    2: (-8, 8)}

for grad_idx in range(3):
    gradients = [None] * len(datas)
    for i, grou in zip(range(2), ["co", "ti"]): # co and ti
        gradients[i] = map_to_labels(gm.aligned_[i][:, grad_idx], labeling,
                                        mask=mask, fill=np.nan)
        
        ## if you want to plot separately
        # p = Plot(lh, rh, background=(1, 1, 1), brightness=0.7)
        # p.add_layer({
        #             'left': gradients[i][:half_sch_idx],
        #             'right': gradients[i][half_sch_idx:]},
        #             cmap="coolwarm",
        #             color_range=color_range_dict1[grad_idx],
        #             as_outline=False,
        #             cbar=True
        #             )
        # p.add_layer({
        #             'left': lh_parc,
        #             'right': rh_parc
        #             },
        #             cmap=cmap,
        #             as_outline=True,
        #             cbar=False
        #             )

        # fig = p.build()
        # fig.show()
        # fig.savefig(ssa_dir.parent / "paper_figures" / "gradients" / f"brain_{grou}_g{grad_idx + 1}.pdf",
        #             dpi=600, bbox_inches='tight')

    ## if you want to plot differences
    diff_grad = gradients[1] - gradients[0] # TI - CO
    p = Plot(lh, rh, background=(1, 1, 1), brightness=0.7)
    p.add_layer({
                'left': diff_grad[:half_sch_idx],
                'right': diff_grad[half_sch_idx:]},
                cmap="coolwarm",
                color_range=(-1, 1), #color_range_dict2[grad_idx],
                as_outline=False,
                cbar=True
                )
    p.add_layer({
                'left': lh_parc,
                'right': rh_parc
                },
                cmap=cmap,
                as_outline=True,
                cbar=False
                )

    fig = p.build()
    fig.show()
    fig.savefig(ssa_dir.parent / "paper_figures" / "gradients" / f"brain_diff_g{grad_idx + 1}.pdf",
                dpi=600, bbox_inches='tight')

In [ ]:
## plot only DMN and somato network on brain
lh, rh = surfaces['inflated']
lh_parc, rh_parc = load_parcellation('schaefer', scale=1000)

lh_parc_somato = np.where((lh_parc >= 82) & (lh_parc <= 172), 1, 0)
rh_parc_somato = np.where((rh_parc >= 582) & (rh_parc <= 684), 1, 0)

lh_parc_dmn = np.where((lh_parc >= 375) & (lh_parc <= 500), 1, 0)
rh_parc_dmn = np.where((rh_parc >= 913) & (rh_parc <= 998), 1, 0)

cl1 = sns.cubehelix_palette(rot=-.2).as_hex()[2]
cl2 = sns.cubehelix_palette(rot=.2).as_hex()[2]
pal1 = [cl1, cl2]


p = Plot(lh, rh, background=(1, 1, 1), brightness=0.7)
p.add_layer({
            'left': lh_parc_dmn,
            'right': rh_parc_dmn,},
            cmap=ListedColormap([cl1]),
            color_range=(0, 1),
            as_outline=False,
            cbar=False
            )
p.add_layer({
            'left': lh_parc_somato,
            'right': rh_parc_somato},
            cmap=ListedColormap([cl2]),
            color_range=(-1, 1),
            as_outline=False,
            cbar=False
            )
p.add_layer({
            'left': lh_parc,
            'right': rh_parc
            },
            cmap=cmap,
            as_outline=True,
            cbar=False
            )
fig = p.build()
fig.show()
fig.savefig(ssa_dir.parent / "paper_figures" / "gradients" / f"brain_networks.pdf",
                dpi=600, bbox_inches='tight')

In [ ]:
## z plots
mode = "2"
df_mds = pd.read_csv(ssa_dir / "zscore_mds.csv")
fname_subjects = "./material/with_qc/behavioural/tinception_fsgd_qc.txt"
df = pd.read_csv(fname_subjects, delimiter="\t")
df.dropna(inplace=True)
df.rename(columns={
                    "Unnamed: 1": "subjects",
                    "Unnamed: 2": "group",
                    "Unnamed: 4": "age",
                    "Unnamed: 7": "PTA"},
                    inplace=True)
subjects_to_drop = [
                    "3xo04l_TI-HA", "S16_hwEa_MRT",
                    "2kyuoc_TI-HL", "3s9x2q_NT-HL",
                    "3ucbt4_NT-HL", "4a71bn_TI-HA",
                    "6ddsfu_NT-HL", "71rpqm_NT-HL",
                    "7jindx_TI-HA", "8i29bt_TI-HA"
                    ]

df = df.query('subjects != @subjects_to_drop')
df = df[["subjects", "group", "age", "PTA"]]
subjects = df["subjects"].values.tolist()
df_mds["subject_id"] = subjects * 3
df_mds["PTA"] = df["PTA"].values.tolist() * 3
df_mds["age"] = df["age"].values.tolist() * 3
df_mds.drop(columns="Unnamed: 0", inplace=True, errors="ignore")

sns.set_theme(style="white")
rs = np.random.RandomState(50)
g = sns.FacetGrid(df_mds, col="gradient_number", row="group", height=3.6)
cmap = sns.cubehelix_palette(start=2.6, light=1, as_cmap=True)

if mode == "1":
    g.map_dataframe(
                    sns.kdeplot,
                    x="X",
                    y="Y",
                    fill=True,
                    cmap=cmap,
                    clip=(-50, 50),
                    cut=5,
                    thresh=0.05,
                    levels=45,
                    )
    g.map_dataframe(
                    sns.scatterplot,
                    x="X",
                    y="Y",
                    color="whitesmoke",
                    s=6,
                    edgecolors="k"
                    )
    
if mode == "2":
    g.map_dataframe(
                    sns.kdeplot,
                    x="X",
                    y="Y",
                    fill=True,
                    cmap=cmap,
                    clip=(-50, 50),
                    cut=5,
                    thresh=0.7,
                    levels=45,
                    )
    for ax in g.axes.flat:
        ax.set_facecolor("lightgrey")
    g.fig.patch.set_facecolor('lightgrey')

for i, ax in enumerate(g.axes.flat):
    row_idx, col_idx = divmod(i, g.axes.shape[1])

    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    if mode == "1":
        if col_idx == 0:
            ax.spines['left'].set_visible(True)
        if row_idx == g.axes.shape[0] - 1:
            ax.spines['bottom'].set_visible(True)

g.set_titles("")
if mode == "2":
    g.set(xticks=[], yticks=[], xlabel="", ylabel="")
plt.tight_layout()
plt.show()
# g.savefig(ssa_dir.parent / "paper_figures" / "gradients" / "mds_2.pdf",
#                     dpi=600, bbox_inches='tight')

In [ ]:
## select some subjects (average deviation maps)
mode = "individual"

fname_subjects = "./material/with_qc/behavioural/tinception_fsgd_qc.txt"
df = pd.read_csv(fname_subjects, delimiter="\t")
df.dropna(inplace=True)
df.rename(columns={"Unnamed: 1": "subjects", "Unnamed: 2": "group"}, inplace=True)
subjects_to_drop = [
                    "3xo04l_TI-HA", "S16_hwEa_MRT",
                    "2kyuoc_TI-HL", "3s9x2q_NT-HL",
                    "3ucbt4_NT-HL", "4a71bn_TI-HA",
                    "6ddsfu_NT-HL", "71rpqm_NT-HL",
                    "7jindx_TI-HA", "8i29bt_TI-HA"
                    ]

df = df.query('subjects != @subjects_to_drop')
df = df[["subjects", "group"]]
subjects = df["subjects"].values.tolist()
zscores = np.load(ssa_dir / "zscores.npy", allow_pickle=True)

if mode == "average":
    df_sub_g1 = df_mds.query('gradient_number == 1 & group == "tinnitus" & -32 < X < -13 & -12 < Y < 10')
    df_sub_g2 = df_mds.query('gradient_number == 2 & group == "tinnitus" & -1.5 < X < 19 & -29 < Y < -16')
    df_sub_g3 = df_mds.query('gradient_number == 3 & group == "tinnitus" & -34 < X < -19 & -9.5 < Y < 18')

if mode == "individual":
    df_sub_g1 = df_mds.query('gradient_number == 1 & group == "tinnitus" & 45 < Y')
    df_sub_g2 = df_mds.query('gradient_number == 2 & group == "tinnitus" & -40 > Y')
    df_sub_g3 = df_mds.query('gradient_number == 3 & group == "tinnitus" & 50 < Y')

labeling = load_parcellation('schaefer', scale=1000, join=True)
mask = labeling != 0
half_sch_idx = 32492
surfaces = fetch_fslr()
lh, rh = surfaces['inflated']
lh_parc, rh_parc = load_parcellation('schaefer', scale=1000)
color = 'dimgrey'
cmap = ListedColormap(color, 'regions', N=1)

for g_idx, df_sel in enumerate([df_sub_g1, df_sub_g2, df_sub_g3]):
    ## compute avg z-score
    subjects_sel = df_sel["subject_id"].values.tolist()
    indices = np.array([subjects.index(item) for item in subjects_sel])
    avg_zscore = zscores[indices, :, g_idx].mean(axis=0)

    ## plot it
    gradients = [None] * len(datas)
    gradients = map_to_labels(avg_zscore, labeling, mask=mask, fill=np.nan)

    p = Plot(lh, rh, background=(1, 1, 1), brightness=0.7)
    p.add_layer({
                'left': gradients[:half_sch_idx],
                'right': gradients[half_sch_idx:]},
                cmap="coolwarm",
                color_range=(-0.6, 0.6), #color_range_dict[g_idx],
                as_outline=False,
                cbar=True
                )
    p.add_layer({
                'left': lh_parc,
                'right': rh_parc
                },
                cmap=cmap,
                as_outline=True,
                cbar=False
                )
    fig = p.build()
    fig.savefig(ssa_dir.parent / "paper_figures" / "gradients" / f"zscores_brain_tin_g{g_idx + 1}_individual.pdf",
                    dpi=600, bbox_inches='tight')


Plot interindividual difference (matrices)

In [ ]:
grad_idx = 0
zscores = np.load(ssa_dir / "zscores.npy", allow_pickle=True)
dist_matrix = squareform(pdist(zscores[:, :, grad_idx], metric='euclidean'))


fname_subjects = "./material/with_qc/behavioural/tinception_fsgd_qc.txt"
df = pd.read_csv(fname_subjects, delimiter="\t")
df.dropna(inplace=True)
df.rename(columns={"Unnamed: 1": "subjects", "Unnamed: 2": "group", "Unnamed: 4": "age", "Unnamed: 7": "PTA"}, inplace=True)
subjects_to_drop = [
                    "3xo04l_TI-HA", "S16_hwEa_MRT",
                    "2kyuoc_TI-HL", "3s9x2q_NT-HL",
                    "3ucbt4_NT-HL", "4a71bn_TI-HA",
                    "6ddsfu_NT-HL", "71rpqm_NT-HL",
                    "7jindx_TI-HA", "8i29bt_TI-HA"
                    ]

df = df.query('subjects != @subjects_to_drop')
df = df[["subjects", "group", "age", "PTA"]]
df = df.reset_index(drop=True)

df['group'] = pd.Categorical(df['group'], categories=['CO', 'TI'], ordered=True)
df_sorted = df.sort_values(by=['group', 'PTA']).reset_index(drop=False)
new_order = df_sorted['index'].values
matrix_sorted = dist_matrix[np.ix_(new_order, new_order)]

## plotting
cm1 = Colormap('colorbrewer:Greys_9')
cm2 = Colormap('colorbrewer:OrRd_9')
colors1 = cm1(np.linspace(0, 1, 5))
colors1 = np.flipud(colors1)
colors2 = cm2(np.linspace(0, 1, 5))  

combined_colors = np.vstack((colors1, colors2))
combined_cmap = LinearSegmentedColormap.from_list("", combined_colors)

fig, ax = plt.subplots(1, 1, figsize=(6, 6), layout="tight")
sns.heatmap(matrix_sorted / matrix_sorted.max(),
            vmin=0.5, vmax=0.8,
            cmap=combined_cmap,
            square=False,
            robust=True,
            linewidths=0.0,
            cbar_kws={"shrink": .7},
            ax=ax,
            cbar=False
            )
ax.axvline(x=129, ymin=0, ymax=1, color="k", linewidth=2)
ax.axhline(y=129, xmin=0, xmax=1, color="k", linewidth=2)
ax.set(xticks=[], yticks=[])

fig, ax = plt.subplots(1, 1, figsize=(6, 2), layout="tight")
sns.lineplot(data=df_sorted,
            x="subjects",
            y="PTA",
            ax=ax,
            color=sns.diverging_palette(230, 20, as_cmap=False)[0],
            lw=2
            )
ax.set(xticks=[], yticks=[], ylabel="", xlim=(0, 256))
ax.spines[["top", "right"]].set_visible(False)

Supplementary figure 1 (MRI QC)

In [ ]:
my_dict = {"bids_id": [], "cjv": [], "cnr": [],
            'snr_csf': [], 'snr_gm': [],
            'snr_total': [], 'snr_wm': []
            }

qc_dir = Path("/Users/payamsadeghishabestari/antinomics_clean_codes/dvob_processed/sMRI/qc_dir")
for fname in sorted(os.listdir(qc_dir)):
    if not fname.startswith("."):
        file_path = qc_dir / fname
        if file_path.is_dir():
            df_json = pd.read_json(file_path / "anat" / f"{fname}_T1w.json")
            my_dict["bids_id"].append(fname)
            for key in list(my_dict.keys())[1:]:
                my_dict[key].append(df_json[key].unique()[0])

df = pd.DataFrame(my_dict)
df = pd.melt(df,
                id_vars=['bids_id'],
                value_vars=['cjv', 'cnr', 'snr_csf', 'snr_gm', 'snr_total', 'snr_wm'],
                var_name='method',
                value_name='value')

df_master = pd.read_excel("/Users/payamsadeghishabestari/tinception/material/behavioural/tinception_master.xlsx")
df["subject_id"] = df_master["subject ID"].values.tolist() * 6
df["group"] = df_master["group"].values.tolist() * 6


g = sns.FacetGrid(data=df, col="method",
                    col_order=["cjv", "cnr", "snr_total"],
                    hue="group", palette=palette_color,
                    sharex=False, sharey=True, legend_out=True)
g.map_dataframe(
                sns.histplot,
                x="value",
                fill=True,
                bins=30,
                alpha=0.7, 
)
g.add_legend()


Supplementary figure 2 (PTA corr with CT)

In [ ]:
fsaverage_dir = Path("/Applications/freesurfer/dev/subjects/fsaverage")
root_dir = Path("./material/with_qc/SBM")

labels, ctab, names = fsio.read_annot(fsaverage_dir / "label" / f"rh.aparc.annot")

column_names = [
    'ClusterNo', 'Max', 'VtxMax', 'Size_mm2', 
    'MNIX', 'MNIY', 'MNIZ', 
    'CWP', 'CWPLow', 'CWPHi', 
    'NVtxs', 'WghtVtx', 'Annot'
]
modality = "area"

for hemi, he in zip(["left", "right"], ["lh", "rh"]):
    sulc = fsaverage_dir / "surf" / f"{he}.sulc"
    inflated = fsaverage_dir / "surf" / f"{he}.inflated"
    
    for mod in ["pos", "neg"]:
        stat_map = root_dir / modality / he / "CO-TI-corr-PTA" / f"perm.th23.{mod}.sig.cluster.mgz"
        summery_fname = root_dir / modality / he / "CO-TI-corr-PTA" / f"perm.th23.{mod}.sig.cluster.summary"

        if mod == "pos":
            input_fname = stat_map

        if mod == "neg": 
            img = nib.load(stat_map)
            data = img.get_fdata()
            data_neg = data * -1
            new_img = nib.Nifti1Image(data_neg, img.affine, img.header)
            input_fname = stat_map.parent / f"perm.th23.{mod}.sig.cluster_inverted.mgz"
            nib.save(new_img, input_fname)
            
        for view in ["medial", "lateral"]:
            figure = plot_surf_stat_map(
                                        stat_map=input_fname,
                                        surf_mesh=inflated,
                                        hemi=hemi,
                                        view=view,
                                        threshold=1.3,
                                        bg_map=sulc,
                                        darkness=0.7,
                                        bg_on_data=False,
                                        vmin=1.3,
                                        vmax=3.89,
                                        colorbar=False,
                                        cmap="autumn"
                                    )
            
            ## if you want to add atlas -> uncomment
            '''
            df = pd.read_csv(summery_fname, skiprows=40, delim_whitespace=True,
                            comment='#', names=column_names)
            labels = df["Annot"].values.tolist()
            levels = [names.index(f"{item}".encode()) for item in labels]

            plot_surf_contours(
                                surf_mesh=fsaverage_dir / "surf" / f"{he}.inflated",
                                roi_map=fsaverage_dir / "label" / f"{he}.aparc.annot",
                                hemi=hemi,
                                labels=labels,
                                levels=levels,
                                legend=False,
                                figure=figure,
                                colors=["cadetblue"] * len(labels),
                            )
            '''

            # figure.figure.set_facecolor('black')
            # figure.axes[0].set_facecolor('black')
            figure.frameon = False
            figure.axes[0].axis('off')
            figure.figure.savefig(Path.cwd() / "material" / "with_qc" / "paper_figures" / "supp_figures" / f"sbm_sig_{modality}_{mod}_{hemi}_{view}.pdf",
                                facecolor=figure.figure.get_facecolor(),
                                bbox_inches='tight')
            show()

Supplementary figure 3 (Lambda)

In [ ]:
## plot lambda scree plot
fname_subjects = "./material/with_qc/behavioural/tinception_fsgd_qc.txt"
df = pd.read_csv(fname_subjects, delimiter="\t")
df.dropna(inplace=True)
df.rename(columns={"Unnamed: 1": "subjects", "Unnamed: 2": "group"}, inplace=True)
subjects_to_drop = [
                    "3xo04l_TI-HA", "S16_hwEa_MRT",
                    "2kyuoc_TI-HL", "3s9x2q_NT-HL",
                    "3ucbt4_NT-HL", "4a71bn_TI-HA",
                    "6ddsfu_NT-HL", "71rpqm_NT-HL",
                    "7jindx_TI-HA", "8i29bt_TI-HA"
                    ]

df = df.query('subjects != @subjects_to_drop')
df = df[["subjects", "group"]]
subjects = df["subjects"].values.tolist()

df.reset_index(drop="index", inplace=True)
co_idxs = df[df["group"] == "CO"].index.to_list()
ti_idxs = df[df["group"] == "TI"].index.to_list()

## plot lambda
ssa_dir = Path("./material/with_qc/SSA")
dfs_list = []
for method in ["dm", "pca"]:

    lambda_data = np.load(ssa_dir / f"lambda_1000_{method}.npy", allow_pickle=True)
    if method == "dm":
        lambda_data = lambda_data / lambda_data.sum(axis=1, keepdims=True)

    df = pd.DataFrame(lambda_data)
    df['group'] = 'unknown'
    df.loc[co_idxs, 'group'] = 'Control'
    df.loc[ti_idxs, 'group'] = 'Tinnitus'

    df_long = df.reset_index().melt(id_vars=['index', 'group'], var_name='Time', value_name='Eigenvalue')
    df_long.rename(columns={'index': 'Subject'}, inplace=True)
    df_long['Gradient'] = df_long['Time'].astype(int) + 1
    df_long["method"] = method
    dfs_list.append(df_long)

df_lambda = pd.concat(dfs_list)

palette_color = ['#1f77b4', '#d62728']
g = sns.FacetGrid(data=df_lambda, col="method", sharex=True,
                    sharey=True, aspect=1.2, height=4, legend_out=False)
g.map_dataframe(
                sns.boxplot,
                x='Gradient',
                y='Eigenvalue',
                hue="group",
                dodge=True,
                width=0.8,
                gap=0.2,
                color="white",
                fill=False,
                palette=palette_color,
                )
g.map_dataframe(
                sns.stripplot,
                x='Gradient',
                y='Eigenvalue',
                hue="group",
                dodge=True,
                # width=0.8,
                # gap=0.2,
                color="white",
                s=1,
                # fill=False,
                palette=palette_color,
                )
g.add_legend()
g.refline(y=0.1)


Supplementary figure 4 (PTA corr with MDS)

In [ ]:
## correlation plot mds vs PTA

df = df_mds.copy()
def get_residuals(y, covariate):
    covariate = sm.add_constant(covariate)
    model = sm.OLS(y, covariate).fit()
    return model.resid

df['X_resid'] = get_residuals(df['X'], df['age'])
df['PTA_resid'] = get_residuals(df['PTA'], df['age'])

results = []
for (grad, group), subset in df.groupby(['gradient_number', 'group']):
    X = sm.add_constant(subset['X_resid'])
    y = subset['PTA_resid']
    model = sm.OLS(y, X).fit()
    r2 = model.rsquared
    pval = model.pvalues[1]  # Coefficient p-value for X_resid
    results.append({'gradient_number': grad, 'group': group, 'r2': r2, 'pval': pval})

stats_df = pd.DataFrame(results)
palette_color = ['#1f77b4', '#d62728']
g = sns.lmplot(
    data=df, x='X_resid', y='PTA_resid', col='gradient_number', hue='group',
    height=4, aspect=1, palette=palette_color, scatter_kws={"s": 7}
)
for i, grad in enumerate(g.col_names):
    ax = g.axes[0, i]
    for group in df['group'].unique():
        stat = stats_df[(stats_df['gradient_number'] == grad) & (stats_df['group'] == group)]
        if not stat.empty:
            r2 = stat['r2'].values[0]
            pval = stat['pval'].values[0]
            ax.text(0.05, 0.95 - 0.1 * list(df['group'].unique()).index(group), 
                    f'{group}: $R^2$={r2:.3f}, p={pval:.3f}', 
                    transform=ax.transAxes, fontsize=9, verticalalignment='top')
plt.show()